In [7]:
import sys
import os
# Add 'src' directory to Python path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [8]:

from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from src import helper_functions as hf
# from src import schema as sc   # your custom schema function
# from src import datacleaning as dcp
# from src import hf_analysis as hf
# from src import franchise_analysis as franch
# from src import visualisation as visn
from pyspark.sql.types import *
from pyspark.sql.functions import * 



In [ ]:
# Initialize Spark
spark = SparkSession.builder.appName("streaming_data").getOrCreate()

''' Define struct type of columns in dataframe 
read the csv files 

'''

# Load JSON data using the schema
# json_path = "/Users/gyauk/github/labs/Pyspark_IMBD_movie_analysis/data/raw/movies.json"
# df = spark.read.schema(hf.schema_build()).json(json_path)
